# Ground Truth Estimation

> Notebook to derive reference segmentations from segmentations of multiple experts (Based on [SimpleITK](http://insightsoftwareconsortium.github.io/SimpleITK-Notebooks/Python_html/34_Segmentation_Evaluation.html)).

For more information on ground truth estimation methods see _Biancardi, Alberto M., Artit C. Jirapatnakul, and Anthony P. Reeves. "A comparison of ground truth estimation methods." International journal of computer assisted radiology and surgery 5.3 (2010): 295-305_.

In [6]:
#@title Set up Google Colab environment
#@markdown Please run this cell to get started.
try:
  from google.colab import files, drive
  !pip install -q simpleITK deepflash2
except:
  pass
import zipfile
import imageio
import SimpleITK as sitk
from fastai.vision.all import *
from deepflash2.data import _read_msk

def staple(segmentations, foregroundValue = 1, threshold = 0.5):
    'STAPLE: Simultaneous Truth and Performance Level Estimation with simple ITK'
    segmentations = [sitk.GetImageFromArray(x) for x in segmentations]
    STAPLE_probabilities = sitk.STAPLE(segmentations) 
    STAPLE = STAPLE_probabilities > threshold
    return sitk.GetArrayViewFromImage(STAPLE)

def mvoting(segmentations, labelForUndecidedPixels = 0):
    'Majority Voting from sitk.LabelVoting'
    segmentations = [sitk.GetImageFromArray(x) for x in segmentations]
    mv_segmentation = sitk.LabelVoting(segmentations, labelForUndecidedPixels)  
    return sitk.GetArrayViewFromImage(mv_segmentation)

## Provide Reference Segmentations from different experts

- __One folder per expert__
- __Identical names for segmentations__

_Examplary structure:_

* [folder] expert1
  * [file] mask1.png
  * [file] mask2.png
* [folder] expert1
  * [file] mask1.png
  * [file] mask2.png

You can either upload a *zip* folder or connect to your _Google Drive_.

### Upload _zip_ file

- The *zip* file must contain all segmentations and correct folder structure. 
- See [here](https://www.hellotech.com/guide/for/how-to-zip-a-file-mac-windows-pc) how to _zip_ files on Windows or Mac.

In [2]:
#@markdown Run to upload a *zip* file
path = Path('expert_segmentations')
u_dict = files.upload()
for key in u_dict.keys():
  zip_ref = zipfile.ZipFile(key, 'r')
  zip_ref.extractall(path)
  zip_ref.close()
masks = get_image_files(path)
experts = set([m.parent.name for m in masks])
print(f'You have uploaded {len(masks)} files from the following experts: {experts}')

Saving Archive.zip to Archive.zip
You have uploaded 80 files from the following experts: {'masks_DS', 'masks_RB', 'masks_AS', 'masks_NS', 'masks_RG'}


### Connect to _Google Drive_

- The folder in your drive must contain all segmentations and correct folder structure. 
- See [here](https://support.google.com/drive/answer/2375091?co=GENIE.Platform%3DDesktop&hl=en) how to organize your files in _Google Drive_.

In [3]:
#@markdown Provide the path to the folder on your _Google Drive_
drive.mount('/content/drive')
path = "/content/drive/My Drive/expert_segmentations" #@param {type:"string"}
path = Path(path)
#@markdown Example: "/content/drive/My Drive/expert_segmentations"
masks = get_image_files(path)
experts = set([m.parent.name for m in masks])
print(f'You have provided {len(masks)} files from the following experts: {experts}')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
You have provided 80 files from the following experts: {'masks_DS', 'masks_RB', 'masks_AS', 'masks_NS', 'masks_RG'}


## Simultaneous truth and performance level estimation (STAPLE)

The STAPLE algorithm considers a collection of segmentations and computes a probabilistic estimate of the true segmentation and a measure of the performance level represented by each segmentation. 

_Source: Warfield, Simon K., Kelly H. Zou, and William M. Wells. "Simultaneous truth and performance level estimation (STAPLE): an algorithm for the validation of image segmentation." IEEE transactions on medical imaging 23.7 (2004): 903-921_

In [ ]:
#@markdown Run STAPLE
path_staple = path/'staple'
path_staple.mkdir(exist_ok=True)
unique_masks = set([m.name for m in masks])
for msk_name in progress_bar(unique_masks):
  print('Processing', msk_name)
  segmentations = [_read_msk(m) for m in masks if m.name==msk_name]
  staple_segmentation = staple(segmentations)
  imageio.imsave(path_staple/msk_name, staple_segmentation*255 if staple_segmentation.max()==1 else staple_segmentation)

If connected, the ground truth estimations are automatically added to you Google Drive. You can also download the files here:

In [ ]:
#@markdown Download STAPLE results
# create a ZipFile object
zipObj = zipfile.ZipFile('staple_export.zip', 'w')
# Add files to the zip
for f in get_image_files(path_staple):
  zipObj.write(f)
# close the Zip File
zipObj.close()
# download files
files.download('staple_export.zip')

## Majority Voting
Use majority voting to obtain the reference segmentation. Note that this filter does not resolve ties. In case of ties it will assign `labelForUndecidedPixels` to the result. 

In [ ]:
#@markdown Run Majority Voting
labelForUndecidedPixels = 0 #@param {type:"integer"}
path_mv = path/'mv'
path_mv.mkdir(exist_ok=True)
unique_masks = set([m.name for m in masks])
for msk_name in progress_bar(unique_masks):
  print('Processing', msk_name)
  segmentations = [_read_msk(m) for m in masks if m.name==msk_name]
  mv_segmentation = mvoting(segmentations, labelForUndecidedPixels)
  imageio.imsave(path_mv/msk_name, mv_segmentation*255 if mv_segmentation.max()==1 else mv_segmentation)

If connected, the _ground truth estimations_ are automatically added to you _Google Drive_. You can also download the files here:

In [10]:
#@markdown Download majority voting results
# create a ZipFile object
zipObj = zipfile.ZipFile('mv_export.zip', 'w')
# Add files to the zip
for f in get_image_files(path_mv):
  zipObj.write(f)
# close the Zip File
zipObj.close()
# download files
files.download('mv_export.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>